In [ ]:
import nevergrad as ng
import pandas as pd
import numpy as np
import warnings
import math
from tqdm import tqdm
import multiprocessing as mp

_GLOBAL_EVALUATIONS = []

def objective_function(east_material, east_quantity, east_padding,
                      west_material, west_quantity, west_padding,
                      tail_material, tail_quantity, tail_padding,
                      trimmer_material, header_material, plank_material):
    
    param_dict = {
        'east_material': east_material,
        'east_quantity': east_quantity,
        'east_padding': east_padding,
        'west_material': west_material,
        'west_quantity': west_quantity,
        'west_padding': west_padding,
        'tail_material': tail_material,
        'tail_quantity': tail_quantity,
        'tail_padding': tail_padding,
        'trimmer_material': trimmer_material,
        'header_material': header_material,
        'plank_material': plank_material,
    }
    
    try:
        hyperparams = {
            'east_joists': MemberSpec(east_material, quantity=east_quantity, padding=east_padding),
            'west_joists': MemberSpec(west_material, quantity=west_quantity, padding=west_padding),
            'tail_joists': MemberSpec(tail_material, quantity=tail_quantity, padding=tail_padding),
            'trimmers': MemberSpec(trimmer_material, quantity=2),
            'header': MemberSpec(header_material, quantity=1),
            'planks': MemberSpec(plank_material),
        }

        frame, _, members = create_model(hyperparams, walls=True)
        part_evaluations = evaluate_stresses(frame, members)
        total_cost, cuts = calculate_purchase_quantity(frame, members)
        member_evaluations = group_stresses_by_member(part_evaluations)

        # Optimize to target ratio
        target = 1
        above_target_multiplier = 2.0
        below_target_multiplier = 1.0

        diff = member_evaluations - target
        multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
        member_evaluations_penalty = multiplier * (diff * diff)

        variety_factor = math.log(len(cuts), 99) + 1

        max_ratio = member_evaluations_penalty.max().max()
        mean_ratio = member_evaluations_penalty.mean().mean()
        score = total_cost * mean_ratio * max_ratio * variety_factor

    except Exception as e:
        warnings.warn(f"An exception occurred with params {param_dict}: {e}")
        score = 1e12
        total_cost = float('inf')
        member_evaluations = pd.DataFrame()
        cuts = {}
        max_ratio = float('inf')
        mean_ratio = float('inf')

    run_result = {
        **param_dict,
        'total_cost': total_cost,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'score': score,
        'cuts': cuts,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS.append(run_result)

    return score


def optimize_with_nevergrad(space_config, n_calls=2048, n_workers=1, algorithm='NGOpt'):
    param_dict = {}
    for param_name, param_spec in space_config.items():
        if param_spec['type'] == 'categorical':
            param_dict[param_name] = ng.p.Choice(param_spec['choices'])
        elif param_spec['type'] == 'integer':
            param_dict[param_name] = ng.p.Scalar(lower=param_spec['lower'], upper=param_spec['upper']).set_integer_casting()

    parametrization = ng.p.Instrumentation(**param_dict)
    
    # Select optimizer
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=n_workers)
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function(**x.kwargs)
        optimizer.tell(x, value)
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


# def optimize_with_cma_es(space_config, n_calls=2048):
    import cma
    
    # Extract bounds and create encoding/decoding
    param_names = []
    bounds_lower = []
    bounds_upper = []
    categorical_indices = {}
    
    for i, (param_name, param_spec) in enumerate(space_config.items()):
        param_names.append(param_name)
        
        if param_spec['type'] == 'categorical':
            # Encode categorical as integer index
            categorical_indices[i] = param_spec['choices']
            bounds_lower.append(0)
            bounds_upper.append(len(param_spec['choices']) - 1)
        else:
            bounds_lower.append(param_spec['lower'])
            bounds_upper.append(param_spec['upper'])
    
    def decode_and_evaluate(x):
        """Convert CMA-ES continuous values to actual parameters"""
        params = {}
        for i, (param_name, value) in enumerate(zip(param_names, x)):
            if i in categorical_indices:
                # Round to nearest integer and map to categorical
                idx = int(round(np.clip(value, bounds_lower[i], bounds_upper[i])))
                params[param_name] = categorical_indices[i][idx]
            else:
                # Round to integer
                params[param_name] = int(round(value))
        
        return objective_function(**params)
    
    # Initial guess (middle of bounds)
    x0 = [(l + u) / 2 for l, u in zip(bounds_lower, bounds_upper)]
    
    # Standard deviation (scale of initial exploration)
    sigma0 = 0.3
    
    # CMA-ES options
    opts = {
        'bounds': [bounds_lower, bounds_upper],
        'maxfevals': n_calls,
        'verb_disp': 1,
        'verb_log': 0,
        'popsize': 32,  # Population size, adjust based on dimensionality
    }
    
    print(f"Running CMA-ES optimization with {n_calls} evaluations...")
    
    es = cma.CMAEvolutionStrategy(x0, sigma0, opts)
    
    with tqdm(total=n_calls) as pbar:
        while not es.stop():
            solutions = es.ask()
            fitness_values = [decode_and_evaluate(x) for x in solutions]
            es.tell(solutions, fitness_values)
            pbar.update(len(solutions))
    
    best_x = es.result.xbest
    
    # Decode best solution
    best_params = {}
    for i, (param_name, value) in enumerate(zip(param_names, best_x)):
        if i in categorical_indices:
            idx = int(round(np.clip(value, bounds_lower[i], bounds_upper[i])))
            best_params[param_name] = categorical_indices[i][idx]
        else:
            best_params[param_name] = int(round(value))
    
    return {
        'best_params': best_params,
        'best_score': es.result.fbest,
        'cma_result': es.result
    }


if __name__ == '__main__':
    # Units are mm, N, and MPa (N/mm²)
    INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

    DL_COMBO = 'DL'
    LL_COMBO = 'LL'
    ULS_COMBO = 'ULS_Strength'

    viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
    beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
    double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
    floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

    max_west_padding = math.ceil(INPUT_PARAMS.opening_x_start / 2)
    max_east_padding = math.ceil((INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)) / 2)
    max_tail_padding = math.ceil(INPUT_PARAMS.opening_length / 3)

    beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
    max_west_quantity = math.floor(max_west_padding / beam_max_base)
    max_east_quantity = math.floor(max_east_padding / beam_max_base)
    max_tail_quantity = math.floor(max_tail_padding / beam_max_base)

    space_config = {
        'east_material': {'type': 'categorical', 'choices': beam_ids},
        'east_quantity': {'type': 'integer', 'lower': 1, 'upper': max_east_quantity},
        'east_padding': {'type': 'integer', 'lower': 0, 'upper': max_east_padding},
        
        'west_material': {'type': 'categorical', 'choices': beam_ids},
        'west_quantity': {'type': 'integer', 'lower': 1, 'upper': max_west_quantity},
        'west_padding': {'type': 'integer', 'lower': 0, 'upper': max_west_padding},
        
        'tail_material': {'type': 'categorical', 'choices': beam_ids},
        'tail_quantity': {'type': 'integer', 'lower': 1, 'upper': max_tail_quantity},
        'tail_padding': {'type': 'integer', 'lower': 0, 'upper': max_tail_padding},
        
        'trimmer_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
        'header_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
        'plank_material': {'type': 'categorical', 'choices': floor_ids},
    }

    n_calls = 2048
    _GLOBAL_EVALUATIONS.clear()
    
    result = optimize_with_nevergrad(
        space_config, 
        n_calls=n_calls,
        n_workers=-1,
        algorithm='NGOpt'  # Try also: 'CMA', 'NGOpt', 'TwoPointsDE', 'PSO', 'DE'
    )
    
    # Option 2: Pure CMA-ES (uncomment to use instead)
    # result = optimize_with_cma_es(space_config, n_calls=n_calls)
    
    results_df = pd.DataFrame(_GLOBAL_EVALUATIONS)
results_df.sort_values(by='score')

In [ ]:
# Single run
from floor_generator import *

hyperparams = {
    'east_joists' : MemberSpec('c24_80x160', quantity=1, padding=100),
    'west_joists' : MemberSpec('c24_100x200', quantity=1, padding=200),
    'tail_joists' : MemberSpec('c24_45x75', quantity=3, padding=250),
    'trimmers' : MemberSpec('c24_100x200', quantity=2),
    'header' : MemberSpec('c24_80x160', quantity=1),
    'planks' : MemberSpec('c18_200x25'),
    }

frame, nodes, members = create_model(hyperparams, walls=True)
part_evaluations = evaluate_stresses(frame, members)
total_cost, cuts = calculate_purchase_quantity(frame, members)
member_evaluations = group_stresses_by_member(part_evaluations)
member_evaluations